La ce ne gândim când începem să conturăm arhitectura unei rețele?
- **inițializare**: 
    - *numărul de noduri de input, output și ascunse*. Altfel spus, stabilim forma și dimensiunile rețelei în etapa aceasta. Vom alege să nu hard codăm acești parametri ci să permitem setarea lor pentru fiecare obiect NeuralNetwork în parte.
    - *legături (weights)* - le folosim ca să propagăm înainte semnalul de intrare prin rețea, precum și ca să propagăm înapoi erorile și să actualizăm aportul adus de acele legături la predicția finală. (De revăzut slide-urile din prima parte a workshop-ului pentru o imagine mai clară).
    Cum reprezentăm weight-urile? 
    Ca matrici: 
    *Wih* - matrice no_hidden x no_input, pentru legăturile dintre stratul de intrare și cel ascuns
    *Who* - matrice no_output x no_hidden, pentru legăturile dintre stratul ascuns și cel de ieșire
    
- **antrenare**: rafinarea weight-urilor (parametrii rețelei)
    - aici avem două părți: 
        1. Feed-forward: pasăm input-ul prin rețea până obținem un output.
        2. Backpropagation: calculăm eroarea (diferența) dintre output-ul obținut și ce ne-am fi așteptat să obținem (ground truth) și pe baza acesteia actualizăm parametrii rețelei (weight-urile).
- **testare**: 
    - folosim rețeaua antrenată pe date noi, pe care aceasta nu le-a mai văzut până acum.
    - trebuie executată pasarea înainte (forward pass) a datelor:
        1. input->hidden: X_hidden = W_ih * I: 
        ceea ce intră în stratul ascuns este de fapt input-ul moderat al rețelei, în funcție de cât de puternice sunt legăturile dintre neuronii de pe stratul de intrare și corespondenții de pe stratul ascuns.
        2. hidden->output: O_hidden = sigmoid(X_hidden):
        pur și simplu aplicăm funcția sigmoidală pe input-urile stratului ascuns.
        3. 4. Similar cu 1. & 2. pentru stratul final (de ieșire).

In [1]:
import numpy as np
import scipy.special as ss

In [2]:
class NeuralNetwork:
    
    def __init__(self, no_input, no_hidden, no_output, lr):
        """
        Initialize the neural network: set the number of neurons in each layer and the learning rate.
        """
        self.no_input = no_input
        self.no_output = no_output
        self.no_hidden = no_hidden
        self.lr = lr
        
        # Initialize the weights. They are also an important part of the network.
        # Method 1:
        # rand(r, c) gives a rxc marix filled with numbers randomly picked from the (0, 1) interval. 
        # As the weights can also be negative, we can subtract 0.5 and generate weights in the (-0.5, 0.5) interval.
        self.W_ih =  np.random.rand(no_hidden, no_input) - 0.5
        self.W_ho =  np.random.rand(no_output, no_hidden) - 0.5
        
        # Method 2:
        # Sample from a normal distribution with mean = 0 (centered) and the standard deviation 1/sqrt(incoming links)
        self.W_ih =  np.random.normal(0.0, pow(no_input, - 0.5), (no_hidden, no_input))
        self.W_ho =  np.random.normal(0.0, pow(no_hidden, - 0.5), (no_output, no_hidden))
        
        # The activation function (sigmoid)
        # Note: 
        # Here we have used a shorter way of defining a function. Instead of using def we have used lambda.
        # Lambda gives us an anonymous function whose definition can appear anywhere in the code.
        # We have assigned a name to this definition as we'll want to use it multiple times from now on.
        # expit is the implementation of the sigmoid function defined in scipy.
        self.activation_function = lambda x: ss.expit(x)
        
    def feed_forward(self, inputs):
        
        # Convert the list of inputs into a 2D array
        I = np.array(inputs, ndmin=2).T
        
        # X_hidden = W_ih * I
        X_hidden = np.dot(self.W_ih, I)
        
        # O_hidden = sigmoid(X_hidden): the output of the hidden layer is its input squashed using the sigmoid function
        O_hidden = self.activation_function(X_hidden)
        
        # X_final = W_ho * O_hidden: the signals into the final output layer
        X_final = np.dot(self.W_ho, O_hidden)
        
        # O_final = sigmoid(X_final)
        O_final = self.activation_function(X_final)
        
        return O_final, O_hidden, I
    
    def train(self, inputs, targets):
        # Feed Forward
        O_final, O_hidden, I = self.feed_forward(inputs)
        
        # Backpropagation
        # Convert the list of target values into a 2D array
        T = np.array(targets, ndmin=2).T
        
        # Compute the error: E_output = T - O_final
        E_output = T - O_final
        
        # Compute the backpropagated errors for the hidden layer nodes
        # E_hidden = transposed(W_ho) * E_output
        E_hidden = np.dot(self.W_ho.T, E_output)
        
        # Update the weights between the hidden and output layers using the formula obtained in the first part of the workshop
        # delta_w_jk = lr * E_k *sigmoid(O_k) * (1 - sigmoid(O_k)) * trasposed(O_j)
        self.W_ho += self.lr * np.dot((E_output * O_final * (1 - O_final)), np.transpose(O_hidden))
        
        # Update the weights between the hidden and the input layer
        self.W_ih += self.lr * np.dot((E_hidden * O_hidden * (1 - O_hidden)), np.transpose(I))
        
    
    def test(self, inputs):
        """
        Pass the input of the network through the layers and return the corresponding output.
        """
        O_final, _, _ = self.feed_forward(inputs)               
        return O_final
        

Haideți să ne construim propria rețea - adică o instanță a clasei NeuralNetwork.

In [3]:
# number of neurons on each layer
no_input = 3; no_output = 3; no_hidden = 3

# learning rate
lr = 0.3

# create the neural network
nn = NeuralNetwork(no_input, no_hidden, no_output, lr)

# pick some random numbers to check just to be sure there are no errors
nn.test([1.0, 0.5, -1.5])

array([[0.53824619],
       [0.42680568],
       [0.50312207]])